<a href="https://colab.research.google.com/github/guptapallavi30/ScalableML_1/blob/exploration/TextHypothesisClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

# Upload your Kaggle API key file to "/content/kaggle/kaggle.json"
# Replace "/content/kaggle/kaggle.json" with the path where you placed your Kaggle API key file in Colab.
!mkdir -p ~/.kaggle
!cp "../content/kaggle/kaggle.json" ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Download the dataset
import pandas as pd

# Replace "narendrageek/my-dear-watson-translated-text" with the dataset name you want to use.
!kaggle datasets download -d narendrageek/my-dear-watson-translated-text -p /content

# Unzip the downloaded dataset (if necessary).
import zipfile
with zipfile.ZipFile("/content/my-dear-watson-translated-text.zip", "r") as zip_ref:
    zip_ref.extractall("/content")

# Read the dataset into a pandas DataFrame.
df = pd.read_csv("/content/translated_train_dataset.csv")

# Read the dataset into a pandas DataFrame.
dftest = pd.read_csv("/content/translated_test_dataset.csv")

  0% 0.00/3.10M [00:00<?, ?B/s]
100% 3.10M/3.10M [00:00<00:00, 199MB/s]


In [ ]:
df.head()

,id,premise,hypothesis,lang_abv,language,label,premise_translated,hypothesis_translated
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0,and these comments were considered in formulat...,The rules developed in the interim were put to...
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,fr,French,0,Little things like these make a huge differenc...,I was trying to accomplish something.
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0,you know they can't really defend themselves l...,They can't defend themselves because of their ...
4,86aaa48b45,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,th,Thai,1,In role playing as well The opportunity to exp...,Children can see how different ethnic groups are.


In [ ]:
dftest.head()

,id,premise,hypothesis,lang_abv,language,label,premise_translated,hypothesis_translated
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0,and these comments were considered in formulat...,The rules developed in the interim were put to...
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,fr,French,0,Little things like these make a huge differenc...,I was trying to accomplish something.
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0,you know they can't really defend themselves l...,They can't defend themselves because of their ...
4,86aaa48b45,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,th,Thai,1,In role playing as well The opportunity to exp...,Children can see how different ethnic groups are.


In [ ]:
df_train_shape = df.shape
df_test_shape = dftest.shape
print(df_train_shape, df_test_shape)

(12120, 8) (12120, 8)


In [ ]:
# Combine train and test
df_all = [df, dftest]
df_all_frames = [df, dftest]

df_all = pd.concat(df_all_frames)

In [ ]:
df_all.shape

(24240, 8)

In [ ]:
# List of column names for which you want to get unique values and value counts.
selected_columns = ['language', 'label']

for column in selected_columns:
    value_counts = df_all[column]
    null_values = df_all[column].isnull().sum()
    print(f"Column: {column}")
    print("Null Values:", null_values)
    print("Value Counts:")
    print(value_counts)
    print("="*30)

Column: language
Null Values: 0
Value Counts:
0        English
1        English
2         French
3        English
4           Thai
          ...   
12115    English
12116    English
12117    English
12118    English
12119    English
Name: language, Length: 24240, dtype: object
Column: label
Null Values: 0
Value Counts:
0        0
1        2
2        0
3        0
4        1
        ..
12115    2
12116    0
12117    2
12118    2
12119    0
Name: label, Length: 24240, dtype: int64


FROM [DATASET APPROACH SECTION](https://www.kaggle.com/datasets/narendrageek/my-dear-watson-translated-text):

"*The dataset has multiple languages. Hence, I have translated Non-English sentences to English sentences using the Google Transcription python library*"

English texts heavily dominate the language section. For the purpose of simplifying the model, I will choose the English-translated columns as input for the model.

In [ ]:
!pip install langdetect
from langdetect import detect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=44696220a89c730224ed959dc81216edf8ba2bd86fc5057cf0e8d944e32c2c34
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [ ]:
# Verify all entries in 'premise_translated' and 'hypothesis_translated' are in English
def is_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False

# Applying the 'is_english' function to the columns
df_all['is_english_premise_t'] = df_all.premise_translated.apply(is_english)
df_all['is_english_hypothesis_t'] = df_all.hypothesis_translated.apply(is_english)

In [ ]:
all_entries_in_english_p = (df_all.is_english_premise_t == False).sum()
all_entries_in_english_h = (df_all.is_english_hypothesis_t == False).sum()

print(df_all.shape[0])
print("All entries in 'premise_translated' are in English:", all_entries_in_english_p)
print("All entries in 'hypothesis_translated' are in English:", all_entries_in_english_h)

24240
All entries in 'premise_translated' are in English: 4620
All entries in 'hypothesis_translated' are in English: 4685


In [ ]:
print((df_all.lang_abv[(df_all.is_english_premise_t == False) & (df_all.lang_abv != 'en')]).value_counts())
print('-------')
print((df_all.lang_abv[(df_all.is_english_hypothesis_t == False) & (df_all.lang_abv != 'en')]).value_counts())

#.iloc[0]

zh    326
vi    324
sw    316
fr    315
ur    313
ru    313
ar    302
tr    297
hi    296
th    295
el    291
de    274
bg    269
es    254
Name: lang_abv, dtype: int64
-------
zh    352
ar    334
ru    325
ur    323
th    316
hi    311
vi    311
tr    309
fr    307
el    296
sw    290
de    283
es    280
bg    266
Name: lang_abv, dtype: int64


In [ ]:
!pip install googletrans==3.1.0a0
# For all entries that are not in english, convert them to English ourselves
from googletrans import Translator

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 5.8 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16352 sha256=4e25697f6bcf50b411844b70eeb3fac7640e76335f86af2aaaf4d9e6d994b282
  Stored in directory: /root/.cache/pip/wheels/50/5d/3c/8477d0af4ca2b8b1308812c09f1930863caeebc762fe265a95
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 4.0.0
    Uninstalling char

In [ ]:
def translate_to_english(sentence, source_language='auto', target_language='en'):
    translator = Translator()
    translated_text = translator.translate(sentence, src=source_language, dest=target_language)
    return translated_text.text

In [ ]:
# Filtering rows based on the conditions and applying the function
mask = (df_all.is_english_premise_t == False) & (df_all.lang_abv != 'en')
df_all.loc[mask, 'premise_translate_modified'] = df_all.loc[mask, 'premise'].apply(translate_to_english)

print(df_all.loc[mask, 'premise_translate_modified'])

186      vahi baat New York Times ke liye nahi kaha ja ...
325      Hundreds of excursionists who sail to both isl...
341      Forming part of the city walls, the gate was p...
363      Despite a two-year investigation, the FBI was ...
378      It came from an air base that flew over Cuba, ...
                               ...                        
12093    Although there is an ongoing kindness of donat...
12099    But my job was to put parachutes on it and lif...
12106    C-R functions can also be estimated with or wi...
12111    -design, draw and hand sew all those gorgeous ...
12112    We didn't know what U2 was and nobody knew any...
Name: premise_translate_modified, Length: 4185, dtype: object


In [ ]:
# verifying how many passed test
passed_test = df_all.loc[mask, 'premise_translate_modified'].apply(is_english)
remaining = (passed_test == False).sum()
remaining

109

In [ ]:
for i in df_all.loc[mask, 'id'][passed_test == False]:
  print(i + " " + df_all.premise_translate_modified[df_all.id == i])

NameError: ignored

In [ ]:
# Translate the non-English statements with ChatGPT

In [ ]:
# translate premise
num_iters = [(df_all.is_english_premise_t == False).sum(), (df_all.is_english_hypothesis_t == False).sum()]
translating_premise = True

# Applying the 'is_english' function to the 'text_column'
# df_all['test_translating_p'] = df_all.premise.apply(translate_to_english)
# df_all['test_translating_h'] = df_all.hypothesis.apply(translate_to_english)

# for num_iter in num_iters:
for i in (df_all.is_english_premise_t == False):
  # print(i)
  # Example sentence in a foreign language (French)
  foreign_sentence = df_all.premise[(df_all.is_english_premise_t == False) & (df_all.lang_abv != 'en')].iloc[i]

  # Translate the foreign sentence to English
  english_translation = translate_to_english(foreign_sentence)

  print("Foreign Sentence:", foreign_sentence)
  print("English Translation:", english_translation)

TypeError: ignored

In [ ]:
# Applying the 'is_english' function to the 'text_column'
df_all['is_english_premise_t'] = df_all.premise_translated.apply(is_english)
df_all['is_english_hypothesis_t'] = df_all.hypothesis_translated.apply(is_english)

----------

In [ ]:
selected_columns = ['premise_translated','hypothesis_translated','label']

# Reduce the DataFrame to the specific columns using the indexing operator [].
reduced_df_all = df_all[selected_columns]

In [ ]:
reduced_df_all.head()

,premise_translated,hypothesis_translated,label
0,and these comments were considered in formulat...,The rules developed in the interim were put to...,0
1,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,2
2,Little things like these make a huge differenc...,I was trying to accomplish something.,0
3,you know they can't really defend themselves l...,They can't defend themselves because of their ...,0
4,In role playing as well The opportunity to exp...,Children can see how different ethnic groups are.,1


In [ ]:
# Data Processing
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# Step 1: Tokenization
sentences = [reduced_df_all.premise_translated, reduced_df_all.hypothesis_translated]
sentences = pd.concat(sentences)
print(type(sentences), sentences)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)

<class 'pandas.core.series.Series'> 0        and these comments were considered in formulat...
1        These are issues that we wrestle with in pract...
2        Little things like these make a huge differenc...
3        you know they can't really defend themselves l...
4        In role playing as well The opportunity to exp...
                               ...                        
12115    All studies have the same amount of uncertaint...
12116    But there are two kinds of the pleasure of doi...
12117                     It cannot be moved, now or ever.
12118         The model temple complex is at the east end.
12119        Ataturk was the father of the Turkish nation.
Length: 48480, dtype: object


In [ ]:
tokenizer.index_word.get(len(tokenizer.index_word))

'เด็กที่ถูกทารุณและถูกทอดทิ้งส่วนใหญ่เป็นคนกลุ่มน้อย'